<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />
<!--- @wandbcode{course-cicd} -->

In [1]:
import os, wandb

# W&B API for CI/CD

The following are examples of operations you can perform with the `wandb` python client that might be relevant for CI/CD.

Perquisite: the environment variable `WANDB_API_KEY`

In [3]:
assert os.getenv('WANDB_API_KEY'), 'You must set the WANDB_API_KEY environment variable'

## Getting metrics tied to the current production model

We can get the production model from the registry, then get the metrics via lineage from the associated run.

In [24]:
run = wandb.init(job_type="CI/CD")
path = 'av-team/model-registry/'
model_name = 'BDD Semantic Segmentation'
version = 'production'

artifact = run.use_artifact(f'{path}{model_name}:{version}', 
                            type='model')

Get the run that created the model:

In [25]:
run = artifact.logged_by()

In [26]:
def get_iou_metrics(run):
    return {k:v for k,v in run.summary.items() if '_iou' in k}

get_iou_metrics(run)

{'road_iou': 0.8379052798838932,
 'person_iou': 0.13954558401437303,
 'final_traffic_sign_iou': 0.05314886217585242,
 'final_bicycle_iou': 0,
 'final_vehicle_iou': 0.7561426671932289,
 'traffic_light_iou': 0.1546196566812521,
 'vehicle_iou': 0.7598675070628712,
 'final_background_iou': 0.920883500388814,
 'traffic_sign_iou': 0.04609753307660045,
 'final_road_iou': 0.8347413133027304,
 'final_traffic_light_iou': 0.11524698900331648,
 'background_iou': 0.921853196957014,
 'final_person_iou': 0.20271520471610835,
 'bicycle_iou': 0}

## Getting metrics for a run ID

This is useful if you want to get metrics for a particular experiment ID.

In [27]:
run_path = 'av-team/mlops-course-001/wlt1r8k7'
api = wandb.Api()
run = api.run(run_path)

In [28]:
get_iou_metrics(run)

{'vehicle_iou': 0.7519784886198334,
 'final_bicycle_iou': 0,
 'final_background_iou': 0.9194038173285132,
 'final_road_iou': 0.8422501240160545,
 'person_iou': 0,
 'final_person_iou': 0,
 'traffic_sign_iou': 0.0001342732460557234,
 'background_iou': 0.9209043224804184,
 'final_traffic_sign_iou': 0.00013421917992081068,
 'final_traffic_light_iou': 0.09025893958076447,
 'bicycle_iou': 0,
 'traffic_light_iou': 0.07797093928386092,
 'road_iou': 0.8467596086120717,
 'final_vehicle_iou': 0.7519664371203332}

## Getting metrics for a run with a tag

It's better to use the [Mongo API](https://docs.wandb.ai/guides/track/public-api-guide#querying-multiple-runs) instead of downloading and iterating through each run if possible.

The use case for this is if you wanted to compare a particular run with another run that you have tagged, such as "baseline".

In [29]:
api = wandb.Api()
tags= ['candidate']

baseline_runs=api.runs('av-team/mlops-course-001', 
                       {"tags": {"$in": tags}}) # this is the Mongo Client

In [30]:
for run in baseline_runs:
    print(run.name)

scarlet-armadillo-502
fine-sweep-63


## Promote a model to the registry

This is also might be something you could do as part of a CI/CD process.

In [8]:
path = 'av-team/model-registry/BDD Semantic Segmentation'
path = 'mustafakeser/model-registry/BDD Semantic Segmentation'


In [6]:
run_path = 'mustafakeser/mlops-course-001/7o21pvs5' # this is the run id
api = wandb.Api()
run = api.run(run_path)

Get the model from the run and promote it to the registry

In [9]:
art = [a for a in run.logged_artifacts() if a.type == 'model']

if art:
    assert len(art) == 1, 'More then 1 artifact of type model!'
    art[0].link(path, aliases=['new-model-demo'])

We can see a model in the registry like this:

In [10]:
versions = api.artifact_versions('model', path)

In [11]:
for v in versions:
    print(f'Model version: {v.version} with tags: {v.aliases}')

Model version: v0 with tags: ['new-model-demo', 'latest']


See the URL for the model registry:

In [12]:
latest_model = versions[0]

In [13]:
# https://wandb.ai/av-team/registry/model?selectionPath=av-team%2Fmodel-registry%2FBDD+Semantic+Segmentation&version=v3

In [14]:
from urllib.parse import urlencode
query = urlencode({'selectionPath': path, 'version': latest_model.version})
registry_url = f'https://wandb.ai/{latest_model.entity}/registry/model?{query}'
registry_url

'https://wandb.ai/mustafakeser/registry/model?selectionPath=mustafakeser%2Fmodel-registry%2FBDD+Semantic+Segmentation&version=v0'

## Run Comparison

[This notebook](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Report_API_Quickstart.ipynb#scrollTo=C3KKJGepCVW4) documents how to use the wandb python client to create a report.  [These docs](https://docs.wandb.ai/guides/reports) describe how to [create](https://docs.wandb.ai/guides/reports/create-a-report) and [edit](https://docs.wandb.ai/guides/reports/edit-a-report) a report.  Make sure you click on the `Python SDK` tab on any 

In [ ]:
/wandb jfcsggkk

In [5]:
import wandb
import wandb.apis.reports as wr

PROJECT = 'h2o-ai-predict-the-llm-kaggle-competition'
ENTITY = None

report = wr.Report(
    entity=ENTITY,
    project=PROJECT,
    title='Compare Runs',
    description="A demo of comparing runs programatically"
)  
runid = "jfcsggkk"
run_path = f'mustafakeser/h2o-ai-predict-the-llm-kaggle-competition/{runid}' # this is the run id
api = wandb.Api()
run = api.run(run_path)
run_name = run.name
compare_list = ['02-Baseline'] + [run_name]
pg = wr.PanelGrid(
    runsets=[
        wr.Runset(ENTITY, PROJECT, "Run Comparison").set_filters_with_python_expr(f"""Name in {compare_list}""")
    ],
    panels=[
        wr.RunComparer(diff_only='split', layout={'w': 24, 'h': 15}),
    ]
)
report.blocks =  [pg] 
report.save()
report_url = report.url

wandb: Thanks for trying out the Report API!
wandb: For a tutorial, check out https://colab.research.google.com/drive/1CzyJx1nuOS4pdkXa2XPaRQyZdmFmLmXV
wandb: 
wandb: Try out tab completion to see what's available.
wandb:   ∟ everything:    `wr.<tab>`
wandb:       ∟ panels:    `wr.panels.<tab>`
wandb:       ∟ blocks:    `wr.blocks.<tab>`
wandb:       ∟ helpers:   `wr.helpers.<tab>`
wandb:       ∟ templates: `wr.templates.<tab>`
wandb:       
wandb: For bugs/feature requests, please create an issue on github: https://github.com/wandb/wandb/issues


In [6]:
report.url

'https://wandb.ai/mustafakeser/h2o-ai-predict-the-llm-kaggle-competition/reports/Compare-Runs--Vmlldzo2MzEyMTEz'